<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/notebooks/%E3%80%90%E5%AE%8C%E6%88%90%E7%89%88%E3%80%91URL%E3%81%8B%E3%82%89%E9%AB%98%E7%B2%BE%E5%BA%A6%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%EF%BC%86Gemini%E5%87%A6%E7%90%86%E5%AE%9F%E8%A1%8C%EF%BC%88%E3%83%97%E3%83%AC%E3%82%A4%E3%83%AA%E3%82%B9%E3%83%88%E5%AF%BE%E5%BF%9C%E7%89%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### **【完成版】動画URLから高精度文字起こし＆Gemini要約 実行スクリプト**

#以下の3つのセルを上から順番に実行してください。

#### **セル1: 環境構築**

# -------------------------------------------------------------------------------------------
# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# 必要なライブラリとツールをインストール
# yt-dlp: 様々なサイトから動画や音声をダウンロードするツール
# faster-whisper: 高速化されたWhisperモデル
# google-generativeai: Gemini APIを使用するためのライブラリ
# ffmpeg: 動画・音声の処理・変換に必須のツール
print("\n▼ 必要なライブラリとツールのインストール")
!pip install -U yt-dlp -q
!pip install faster-whisper==1.0.3 -q
!pip install google-genai -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q
!apt-get update && apt-get install -y ffmpeg -qq

print("\n✅ 環境構築が完了しました。")
print("次のセルに進んでGoogle Driveに接続してください。")

In [ ]:
#### **セル2: Google Driveへの接続 (任意)**
#文字起こししたテキストファイルをGoogle Driveに保存したい場合に、このセルを実行してGoogle Driveをマウント（接続）してください。

# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")
print("最後のセルに進んで、文字起こしを実行してください。")

In [ ]:
#@title 🚀 URLから高精度文字起こし＆Gemini処理実行（高速化修正版）
#@markdown ### 1. 動画のURLと出力先の設定
#@markdown ---
#@markdown 文字起こししたい動画またはプレイリストのURLを入力してください。
video_url = "" #@param {type:"string"}

#@markdown 文字起こし結果（.txtファイル）を保存するフォルダのパスを指定します。
output_transcript_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown **プレイリスト処理**: URLがプレイリストの場合、チェックを入れるとリスト内の全動画を順番に処理します。
enable_playlist = False #@param {type:"boolean"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `Zoont/...-int8-ct2`は精度を維持しつつ最速・省メモリな**総合推奨モデル**です。
model_name = "Zoont/faster-whisper-large-v3-turbo-int8-ct2" #@param ["Zoont/faster-whisper-large-v3-turbo-int8-ct2", "deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium", "small", "base", "tiny"]
#@markdown **計算タイプ**: `int8`モデルには`int8_float16`、`float16`モデルには`float16`の組み合わせを推奨します。
compute_type = "int8_float16" #@param ["int8_float16", "float16", "int8", "float32"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 200 #@param {type:"slider", min:100, max:2000, step:100}

#@markdown ### 4. 言語設定（オプション）
#@markdown ---
#@markdown 文字起こしする言語を事前に指定する場合に有効にします。指定しない場合は、言語は自動で検出されます。
enable_language_specification = False #@param {type:"boolean"}
#@markdown [言語コード一覧](https://github.com/openai/whisper/blob/main/whisper/tokenizer.py)を参考に、2文字の言語コードを入力してください。（例: ja, en, zh）
language_code = "ja" #@param {type:"string"}

#@markdown ### 5. 高度な設定（オプション）
#@markdown ---
#@markdown **ビームサイズ (beam_size)**: 文字起こしの精度と速度のトレードオフを調整します。
beam_size = 5 #@param {type:"slider", min:1, max:10, step:1}
#@markdown **中間ファイルのクリーンアップ**: チェックを入れると、処理終了後にダウンロードした音声ファイルを自動で削除します。
cleanup_audio_file = True #@param {type:"boolean"}

#@markdown ### 6. Geminiによる処理の設定（オプション）
#@markdown ---
#@markdown Geminiを使った処理を有効にする場合は、チェックを入れてAPIキー等を設定してください。
enable_gemini_processing = False #@param {type:"boolean"}
gemini_api_key = "" #@param {type:"string"}
#@markdown **★★★ Geminiモデルをここで選択 ★★★**
gemini_model = "gemini-2.5-flash" #@param ["gemini-2.5-pro", "gemini-2.5-flash", "gemini-2.5-flash-lite"]
#@markdown Geminiの処理結果を保存するフォルダのパスを指定します。
output_gemini_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/gemini_outputs" #@param {type:"string"}
#@markdown **プロンプト (指示内容)**: Geminiに実行させたいタスクを具体的に入力してください。（例：この内容を箇条書きで3点にまとめて。）
gemini_prompt = "以下の動画書き起こしテキストを、重要なポイントと動画の構成を含めて要約して最大コンテクストで出力してください。" #@param {type:"string"}


# --- ここから下は実行コード（編集不要） ---
import os
import yt_dlp
from faster_whisper import WhisperModel
import time
from datetime import datetime, timezone, timedelta
import google.generativeai as genai
import shutil

def get_current_timestamp():
    """現在時刻を[YYYY-MM-DD HH:MM:SS]形式の文字列で返す"""
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def process_single_video(video_info, model):
    """単一の動画を処理する関数"""
    video_id = video_info.get('id', 'default_id')
    video_title = video_info.get('title', 'untitled').replace('/', '／')
    video_url = video_info.get('webpage_url')

    if not video_url and video_id != 'default_id':
        video_url = f"https://www.youtube.com/watch?v={video_id}"

    print(f"  - 対象: {video_title}")
    print(f"  - URL: {video_url}")

    temp_audio_dir = "/content/temp_audio"
    os.makedirs(temp_audio_dir, exist_ok=True)

    # 1. 音声ダウンロード
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav', 'preferredquality': '192'}],
        'outtmpl': os.path.join(temp_audio_dir, f'{video_id}.%(ext)s'),
        'quiet': True,
        'nocheckcertificate': True,
        # ▼▼▼【修正】音声ダウンロード時にも 'noplaylist' を適用し、不要な再スキャンを防止 ▼▼▼
        'noplaylist': not enable_playlist,
    }
    audio_file_path = None
    full_transcript_text = ""
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # download=Falseのままで問題ありません。
            # extract_infoはdownload=Trueが指定された場合、またはダウンロードが必要と判断された場合にダウンロードを実行します。
            # ここではpostprocessorsで音声変換を指定しているため、ダウンロードがトリガーされます。
            ydl.extract_info(video_url, download=True)
            audio_file_path = os.path.join(temp_audio_dir, f'{video_id}.wav')

        if not os.path.exists(audio_file_path):
            raise FileNotFoundError("音声ファイルの生成に失敗しました。")

        # 2. 文字起こし
        start_transcribe_time = time.time()
        transcribe_options = {
            "beam_size": beam_size,
            "vad_filter": use_vad_filter,
            "vad_parameters": dict(min_silence_duration_ms=vad_min_silence_duration_ms)
        }
        if enable_language_specification and language_code:
            transcribe_options["language"] = language_code.strip()

        segments, info = model.transcribe(audio_file_path, **transcribe_options)

        if not (enable_language_specification and language_code):
             print(f"    -> 検出言語: {info.language} (確率: {info.language_probability:.2f})")

        # 3. 結果保存
        safe_title = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in video_title).strip()
        output_txt_filename = f"{video_id}_{safe_title[:50]}.txt"
        transcript_output_path = os.path.join(output_transcript_dir, output_txt_filename)

        transcript_lines = []
        with open(transcript_output_path, "w", encoding="utf-8") as f:
            f.write(f"動画タイトル: {video_title}\n")
            f.write(f"URL: {video_url}\n")
            f.write(f"文字起こし実行日時: {get_current_timestamp()}\n")
            f.write(f"モデル: {model_name} ({compute_type})\n")
            if enable_language_specification and language_code:
                f.write(f"指定言語: {language_code}\n")
            else:
                f.write(f"検出言語: {info.language} (確率: {info.language_probability:.2f})\n")
            f.write(f"VADフィルター: {'有効' if use_vad_filter else '無効'}\n\n---\n\n")

            for segment in segments:
                line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                f.write(line)
                transcript_lines.append(segment.text.strip())
            full_transcript_text = "\n".join(transcript_lines)

        end_transcribe_time = time.time()
        print(f"    -> ✅ 文字起こし完了 ({end_transcribe_time - start_transcribe_time:.2f}秒): {os.path.basename(transcript_output_path)}")

        # 4. Gemini処理
        if enable_gemini_processing:
            if not gemini_api_key:
                print("    -> ⚠️ Gemini APIキー未設定のためスキップします。")
            elif not full_transcript_text.strip():
                print("    -> ⚠️ 文字起こし結果が空のためGemini処理をスキップします。")
            else:
                try:
                    model_gemini = genai.GenerativeModel(gemini_model)
                    final_prompt = f"{gemini_prompt}\n\n---\n\n以下が対象のテキストです。\n\n{full_transcript_text}"
                    response = model_gemini.generate_content(final_prompt)
                    gemini_output_text = response.text

                    output_filename = f"{video_id}_{safe_title[:50]}_gemini_output.txt"
                    output_path = os.path.join(output_gemini_dir, output_filename)
                    with open(output_path, "w", encoding="utf-8") as f:
                        f.write(f"■ 元動画タイトル: {video_title}\n")
                        f.write(f"■ URL: {video_url}\n")
                        f.write(f"■ 実行日時: {get_current_timestamp()}\n")
                        f.write(f"■ 使用モデル: {gemini_model}\n\n---\n\n")
                        f.write(gemini_output_text)
                    print(f"    -> ✅ Gemini処理完了: {os.path.basename(output_path)}")
                except Exception as e:
                    print(f"    -> 💥 Gemini APIエラー: {e}")

    except Exception as e:
        print(f"  ❌ この動画の処理中にエラーが発生しました: {e}")
    finally:
        if cleanup_audio_file and audio_file_path and os.path.exists(audio_file_path):
            os.remove(audio_file_path)

def run_pipeline():
    """メインの処理パイプライン"""
    if not video_url:
        print("❌ エラー: 動画のURLが入力されていません。")
        return

    print(f"{get_current_timestamp()} --- 1. パイプライン開始 ---")
    os.makedirs(output_transcript_dir, exist_ok=True)
    if enable_gemini_processing:
        os.makedirs(output_gemini_dir, exist_ok=True)
        if gemini_api_key:
            genai.configure(api_key=gemini_api_key)
        else:
            print("⚠️ Gemini処理が有効ですが、APIキーが設定されていません。")

    temp_audio_dir = "/content/temp_audio"
    if os.path.exists(temp_audio_dir):
        shutil.rmtree(temp_audio_dir)
    os.makedirs(temp_audio_dir, exist_ok=True)

    print(f"\n{get_current_timestamp()} --- 2. Whisperモデル '{model_name}' ({compute_type}) をロード中... ---")
    start_load_time = time.time()
    try:
        model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
    except Exception as e:
        print(f"❌ モデルのロード中にエラーが発生しました: {e}")
        print("   -> ランタイムのタイプがGPUになっているか確認してください。")
        return
    end_load_time = time.time()
    print(f"✅ Whisperモデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

    print(f"\n{get_current_timestamp()} --- 3. 動画情報の取得を開始 ---")
    videos_to_process = []
    try:
        # ▼▼▼【修正】情報取得時に 'noplaylist' を適用し、単一動画モード時の不要なスキャンを防止 ▼▼▼
        ydl_opts = {
            'quiet': True,
            'nocheckcertificate': True,
            'ignoreerrors': True,
            'noplaylist': not enable_playlist,
            'no_warnings': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=False)
            if enable_playlist and 'entries' in info:
                videos_to_process = [entry for entry in info['entries'] if entry] # Noneのエントリを除外
                print(f"✅ プレイリストを検出しました。{len(videos_to_process)}件の動画を処理します。")
            else:
                videos_to_process.append(info)
                print("✅ 単一動画として処理します。")
    except Exception as e:
        print(f"❌ URLからの情報取得に失敗しました: {e}")
        return

    total_videos = len(videos_to_process)
    if total_videos > 0:
        print(f"\n{get_current_timestamp()} --- 4. 文字起こしとGemini処理を開始 ---")
        for i, video_info in enumerate(videos_to_process, 1):
            print(f"\n--- [{i}/{total_videos}] 処理開始 ---")
            process_single_video(video_info, model)

    print(f"\n{get_current_timestamp()} --- 🎉 全ての処理が完了しました ---")

# パイプライン実行
run_pipeline()
